Before running this application, please remove and replace all files under data/inventory and data/updates in order to execute latest changes.

data/inventory
https://admin.shoplineapp.com/admin/waddystore/reports?createdBy=pos
實體店 > 庫存總表
Retail Store > Product Inventory Report

data/updates
https://admin.shoplineapp.com/admin/waddystore/products/import/bulkupload_v2
匯出商品更新檔
Export Product Bulk Update File

In [1]:
import inv_dataset
import browser
import os
import pandas as pd
import csv

In [2]:
inventory_folder = 'data/inventory/'
udpate_folder_path = 'data/updates/'

#convert inventory xls to csv
inventory_files = inv_dataset.Converter()
inventory_files.to_csv(inventory_folder)

#convert updates xls to csv
update_files = inv_dataset.Converter()
update_files.to_csv(udpate_folder_path)

In [3]:
#read inventory csv into dataframe
inventory = inv_dataset.Reader()
inv_df = inventory.get_df(inventory_folder)

In [4]:
'''
back to parent directory if necessery
Use this line: 
os.chdir("../..")

check current directory:
os.getcwd()
'''

os.getcwd()

'c:\\Users\\Waddy Store\\Documents\\Code Project\\shopline_automation'

In [5]:
def PreOrderClose():
    process_list = []
    #Start update procedure
    os.chdir(udpate_folder_path)
    #go thru each bulk update form csv
    for file in os.listdir():
        if file.endswith(".csv"):
            #in each bulk update form csv
            file_path = f"{file}"
            df = pd.read_csv(file_path, index_col=None, header=0)
            df = df.drop(index=0)

            df = df.loc[df['Preorder Product'] == "Y"]
            df = df.loc[df['Status'] == "Y"]
            
            for barcode in df['Barcode']:
                df2 = inv_df.loc[inv_df['商品條碼'] == barcode]
                df2 = df2.loc[df2['預設倉庫'] > 0]
                for bar in df2['商品條碼']:
                    sku_id = df[df['Barcode']== bar]['Product ID (DO NOT EDIT)'].item()
                    varient_id = df[df['Barcode']== bar]['Variant ID (DO NOT EDIT)'].item()
                    has_varient = True
                    if pd.isnull(varient_id):
                        has_varient = False
                    process_list.append([sku_id, bar, has_varient])

    os.chdir("../..") 
    print(process_list) 
    clicker = browser.Action()
    clicker.close_preorder(process_list)   
    print("All Completed, End Task.")    

- You may encounter NoSuchElementException or ElementNotInteractableException, try to run this part of code again

In [6]:
PreOrderClose()

[['5f267f1ed10af7002149cfcc', '4560220556504', False], ['5f3c8c4da7a677003921361e', '4710006290287', False], ['5f3c90916173d6003357ca25', '4710006290218', False], ['5f3c9df00d6366003cfa30f3', '4710006290188', False], ['5f6f2441c727c4001a7f9541', '839289002166', False], ['5fa92235e075fb16c52c0271', '69356464', False], ['5fae41b741f0a200242031cd', '4251460611039', False], ['5ffd80f0a167f4002fe258c1', '4560220557426', False], ['607df5231a9e4900297bf277', '4251460604321', False], ['607df69e46f82d0023b60300', '4251460604338', False], ['60890eafdf84a9001183d563', '4251460603935', False], ['60ae22105ea1580038f50531', '4582593581327', False], ['60c8927e8a2a9b002908e5a8', '4580279018631', False], ['60e681301658610032056b54', '4582593580986', False], ['60f914af9d99680029d006f8', '4580279019829', False], ['6156cc7e3c09480014c7e9dc', '4573423119346', False], ['615d541665741d0030b0de33', '4571355630618', False], ['6167f5e655e2640035465a4d', '839289007154', False], ['617b9d5cb6aae3002367f009', '4251

In [7]:
def PreOrderOpen():
    file = open("search/namelist.csv", "r")
    data = list(csv.reader(file, delimiter=","))
    file.close()
    search_for = [row[0] for row in data]
    process_list = []

    #Start update procedure
    #go thru each bulk update form csv
    os.chdir(udpate_folder_path)
    for file in os.listdir():
        if file.endswith(".csv"):
            #in each bulk update form csv
            file_path = f"{file}"
            update_df = pd.read_csv(file_path, index_col=None, header=0)
            update_df = update_df.drop(index=0)

            df = update_df.loc[update_df['Product Name (Traditional Chinese)'].str.contains('|'.join(search_for), na = False)]

            ''' end conidtion'''

            df = df.loc[df['Preorder Product'] == "N"]
            df = df.loc[df['Status'] == "Y"]

            discon_tag = 'dis'
            df = df.loc[~df['Product Tag'].str.contains(discon_tag, na=False)]

            discon_tag_manual = 'shortage'
            df = df.loc[~df['Product Tag'].str.contains(discon_tag_manual, na=False)]
            
            for barcode in df['Barcode']:
                df2 = inv_df.loc[inv_df['商品條碼'] == barcode]
                df2 = df2.loc[df2['預設倉庫'] <= 0]
                for bar in df2['商品條碼']:
                    sku_id = df[df['Barcode']== bar]['Product ID (DO NOT EDIT)'].item()
                    varient_id = df[df['Barcode']== bar]['Variant ID (DO NOT EDIT)'].item()
                    has_varient = True
                    if pd.isnull(varient_id):
                        has_varient = False
                    process_list.append([sku_id, bar, has_varient])

    print(process_list) 
    clicker = browser.Action()
    clicker.open_preorder(process_list)   
    print("All Completed, End Task.")    

In [8]:
PreOrderOpen()

[['5f34375084897a0033e902b7', '4560220556559', False], ['5f3b79e20898500042d61b27', '4049369015498', False], ['6123d83fe69bb162d2a82a39', '66483760', False], ['61f2250fda390e002354f117', '4571361303162', False], ['62532d413e3a920015ef72d6', '84655527', False], ['62ba8ff70e76860021973eaf', '34875685', False], ['62d274cb903f0f0029c859b1', '80212065', True], ['638ff494f492497f874aff0b', '4251460617284', False]]
Clicker starts
Login sucessful
Now browsing to SKU: 5f34375084897a0033e902b7
Go to Price and Quantity Tab
Checkbox Status is  None
clicking checkbox
Go to Settings Tab
Not yet switched on Preorder Product Setting
Switched on Preorder Product Setting
Typed in Preorder Product Note (English)
Typed in Preorder Product Note (Chinese)
Saved changes, completed
Now browsing to SKU: 5f3b79e20898500042d61b27
Go to Price and Quantity Tab
Checkbox Status is  None
clicking checkbox
Go to Settings Tab
Not yet switched on Preorder Product Setting
Switched on Preorder Product Setting
Typed in Pre